In [27]:
# China

## Dataset Production

In [4]:
#Read Files

import numpy as np
import pandas as pd

def file_reader(path):
    df=pd.DataFrame()
    chunksize = 10 ** 6
    for chunk in pd.read_csv(path, chunksize=chunksize):
        df = df.append(chunk)
    return df
df=file_reader('/Volumes/BECCA/Project/Data/Input/01_Raw data/China_052020.csv') 

/Users/becca/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
#Drop columns, expect 'text' & 'tweet_language' & 'tweet_time'
for i in df:
    if i != 'tweet_text' :
        if i != 'tweet_language':
            if i != 'tweet_time':
                df=df.drop([i], axis=1)

In [6]:
df

,tweet_language,tweet_text,tweet_time
0,zh,郭瘟鬼这个大骗子，打着换汇的幌子通过他的地下钱庄骗钱，张口就来，每天就是在炫耀自己的骗术，殊...,2019-12-12 01:56
1,und,https://t.co/hNm6nUAiE3,2020-01-06 06:05
2,und,https://t.co/UKuYfubjgh,2020-01-07 10:52
3,und,https://t.co/KtpavahflW,2020-01-06 01:24
4,ja,秋 https://t.co/36ISekYyC9,2019-11-05 00:59
...,...,...,...
348603,zh,每一个普通的改变，都将改变普通。因为从决心变得更好那一刻开始，我们就已经与一个全新的自己不期...,2019-11-12 07:08
348604,zh,受黑暴示威影響，本港訪港旅客數字連跌4個月，香港旅遊及發展局公布10月訪港旅客錄得逾331萬...,2019-11-30 01:24
348605,zh,女孩一定要和满眼都是你的人在一起啊,2019-10-01 01:40
348606,zh,你这么可爱，我一定要爱你，我不允許你自己擅自做主,2019-07-25 01:37


In [7]:
#Just use the tweet in English
tweets=[]
for i in range(len(df)):
    if df.tweet_language[i] == 'en':
        l=[]
        l.append(df.tweet_text[i])
        l.append(df.tweet_time[i])
        tweets.append(l)
tweets=pd.DataFrame(tweets, columns=['tweet_text','tweet_time'])
tweets['suspend']=True

In [8]:
tweets

,tweet_text,tweet_time,suspend
0,Necessity is the mother of invention,2019-10-18 21:21,True
1,#病毒 Constantly defying the will of the citize...,2020-03-26 05:09,True
2,#肺炎 #病毒 The priceless life should not be the...,2020-03-28 02:37,True
3,"Want to see the walking dead, but feel a littl...",2019-10-30 05:57,True
4,Maybe should call that Kekong? #HongKong #Keki...,2020-01-03 01:39,True
...,...,...,...
32921,"Look before you leap. First think, then act.",2019-05-22 08:30,True
32922,I was for all intensive purposes addicted to t...,2019-04-21 03:23,True
32923,.Live a noble and honest life.Reviving past ti...,2019-06-17 08:48,True
32924,"her, and she drooped and would have sunk down ...",2019-10-24 01:19,True


## Topic Analysis
-to split file into different topic cluster, then get dataset under each topic

In [80]:
#tweet_text preprocessing
import preprocessor as p

for i,v in enumerate(tweets['tweet_text']):
    tweets.loc[i,'text'] = p.clean(v)
    
import spacy
# spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

import nltk
# nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

# nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

import random
text_data = []
for i in range(len(tweets)):
    tokens = prepare_text_for_lda(tweets.text[i])
    if random.random() > .9: #random select around 1000 text
        text_data.append(tokens)

In [81]:
#topic analysis
import gensim
from gensim import corpora
dictionary = corpora.Dictionary(text_data) 
corpus =[dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

from itertools import chain
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
lda_corpus = ldamodel[corpus]
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
tweet_list=list(tweets.text)

cluster1 = [j for i,j in zip(lda_corpus,tweet_list) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,tweet_list) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,tweet_list) if i[2][1] > threshold]

In [82]:
len(cluster3)

1149

In [83]:
#visualisation of topic distribution
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
import pyLDAvis.gensim
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

pyLDAvis.save_html(lda_display3, 'lda.html')

In [9]:
tweets

,tweet_text,tweet_time,suspend
0,Necessity is the mother of invention,2019-10-18 21:21,True
1,#病毒 Constantly defying the will of the citize...,2020-03-26 05:09,True
2,#肺炎 #病毒 The priceless life should not be the...,2020-03-28 02:37,True
3,"Want to see the walking dead, but feel a littl...",2019-10-30 05:57,True
4,Maybe should call that Kekong? #HongKong #Keki...,2020-01-03 01:39,True
...,...,...,...
32921,"Look before you leap. First think, then act.",2019-05-22 08:30,True
32922,I was for all intensive purposes addicted to t...,2019-04-21 03:23,True
32923,.Live a noble and honest life.Reviving past ti...,2019-06-17 08:48,True
32924,"her, and she drooped and would have sunk down ...",2019-10-24 01:19,True


In [11]:
country='China'
tweets=tweets.drop(['text'], axis=1)
tweets.to_csv('/Volumes/BECCA/Project/Data/Input/02_Preprocessing/01_Suspend %s.csv' %country)

In [86]:
#keyword list
keyword1=[]
keyword2=[]
keyword3=[]
keywords=[keyword1,keyword2,keyword3]

for idx, topic in ldamodel.show_topics(formatted=False, num_words= 2): #2 topics would make api find result much more easiler
    for w in topic:
        if idx==0:
            keyword1.append(w[0])
        elif idx==1:
            keyword2.append(w[0])
        elif idx==2:
            keyword3.append(w[0])

In [87]:
keywords

[['milesguo', 'wengui'], ['milesguo', 'china'], ['world', 'epidemic']]

In [88]:
# Russia

In [12]:
#Read Files

import numpy as np
import pandas as pd

def file_reader(path):
    df=pd.DataFrame()
    chunksize = 10 ** 6
    for chunk in pd.read_csv(path, chunksize=chunksize):
        df = df.append(chunk)
    return df
df=file_reader('/Volumes/BECCA/Project/Data/Input/01_Raw data/Russia_052020.csv') 

/Users/becca/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (0,1,9,16,17,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/becca/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (0,1,9,17,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [13]:
#Drop columns, expect 'text' & 'tweet_language' & 'tweet_time'
for i in df:
    if i != 'tweet_text' :
        if i != 'tweet_language':
            if i != 'tweet_time':
                df=df.drop([i], axis=1)

In [14]:
#Just use the tweet in English
tweets=[]
for i in range(len(df)):
    if df.tweet_language[i] == 'en':
        l=[]
        l.append(df.tweet_text[i])
        l.append(df.tweet_time[i])
        tweets.append(l)
tweets=pd.DataFrame(tweets, columns=['tweet_text','tweet_time'])
tweets['suspend']=True

In [92]:
#tweet_text preprocessing
import preprocessor as p

for i,v in enumerate(tweets['tweet_text']):
    tweets.loc[i,'text'] = p.clean(v)
    
import spacy
# spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

import nltk
# nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

# nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

import random
text_data = []
for i in range(len(tweets)):
    tokens = prepare_text_for_lda(tweets.text[i])
    if random.random() > .9: #random select around 1000 text
        text_data.append(tokens)

In [93]:
#topic analysis
import gensim
from gensim import corpora
dictionary = corpora.Dictionary(text_data) 
corpus =[dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

from itertools import chain
# ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, \
#                                            update_every=1, chunksize=10000, passes=15)
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
lda_corpus = ldamodel[corpus]
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
tweet_list=list(tweets.text)


cluster1 = [j for i,j in zip(lda_corpus,tweet_list) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,tweet_list) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,tweet_list) if i[2][1] > threshold]

In [94]:
#visualisation of topic distribution
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
import pyLDAvis.gensim
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

country='Russia'
pyLDAvis.save_html(lda_display3, 'lda_%s.html' %country)

In [15]:
country='Russia'
tweets=tweets.drop(['text'], axis=1)
tweets.to_csv('/Volumes/BECCA/Project/Data/Input/02_Preprocessing/01_Suspend %s.csv' %country)

In [96]:
#keyword list
keyword1=[]
keyword2=[]
keyword3=[]
keywords=[keyword1,keyword2,keyword3]

for idx, topic in ldamodel.show_topics(formatted=False, num_words= 2): #2 topics would make api find result much more easiler
    for w in topic:
        if idx==0:
            keyword1.append(w[0])
        elif idx==1:
            keyword2.append(w[0])
        elif idx==2:
            keyword3.append(w[0])

In [97]:
keywords

[['people', 'football'], ['girl', 'naked'], ['followers', 'stats']]